# Ch 46 North Dakota

 [The North Dakota Legislative Branch](https://www.ndlegis.gov/legislative-histories/session-laws) stores all session laws of North Dakota since 1862.

![North DakotaLegislature](pics/nd_web.png)


## Import libraries
As introduced in the chapter 1, we need to import some libraries as follows.

In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from time import sleep
import pandas as pd
import datefinder
import calendar
import os
import unittest
from random import randint
import PyPDF2
import glob
import pickle
import numpy as np
import fitz

## Browser setup

Before scraping, we need to set up the browser. Here we use ChromeDriver for Google Chrome. You can download ChromeDriver from https://chromedriver.chromium.org/downloads following the version of Google Chrome you use on your laptop.

In [ ]:
driver_path = 'yourpath/driver'

# Change the working directory to your path on your computer
os.chdir('yourpath')

# Get the working directory
cwd = os.getcwd()
# Print the current working directory
print("Current working directory: {0}".format(cwd))

# Set up the driver
chromeOptions = webdriver.ChromeOptions()
dnldpath = {"download.default_directory": "your_download_path"}
chromeOptions.add_experimental_option('prefs', {
    "download.default_directory": dnldpath,  #Change default directory for downloads
    "download.prompt_for_download": False,  #To auto download the file
    "download.directory_upgrade": True,
    "plugins.always_open_pdf_externally": True,  #It will not show PDF directly in chrome
    "--enable-javascript": True
})
# Open the driver
driver = webdriver.Chrome(executable_path=driver_path, options=chromeOptions)

## PDF file download

In [ ]:
driver.get("https://www.ndlegis.gov/legislative-histories/session-laws")

urls = []
sessionurls = []

sessions = driver.find_elements_by_partial_link_text("Session Laws")
for session in sessions:
    if int(session.text[:4]) > 1974:
        print(session.text)
        sessionurl = session.get_attribute('href')
        print(sessionurl)
        sessionurls.append(sessionurl)
    else:
        pass

for sessionurl in sessionurls:
    driver.get(sessionurl)
    sleep(1)
    driver.find_element_by_link_text("Chapter Categories").click()
    try:
        chapters = driver.find_elements_by_css_selector('div > div > div > table > tbody > tr > td > p > a')
        for chapter in chapters:
            print(chapter.text)
            if "Resolutions" in chapter.text:
                pass
            else:
                url = chapter.get_attribute('href')
                urls.append(url)
                print(url)
    except:
        print(sessionurl)
print("urls are done")

acttxts = []
for idx, url in enumerate(urls):
    driver.get(url)
    sleep(1)
    path = "C:/Users/longy/OneDrive/Projects/AFRI/data/ND/"
    list_of_files = glob.glob(os.path.join(path, '*.pdf'))
    latest_file = max(list_of_files, key=os.path.getctime)
    print(latest_file)
    with open(os.path.join(os.getcwd(), latest_file), 'r') as f:  # open in readonly mode
        # creating a pdf File object of original pdf
        pdfFileObj = open(latest_file, 'rb')
        pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
        pagenumber = pdfReader.numPages
        acttxt = []
        pgtxts = []
        for p in range(pagenumber):
            pageObj = pdfReader.getPage(p)
            pgtxt = pageObj.extractText()
            pgtxts.append(pgtxt)
            acttxt = '\n'.join(pgtxts)
    acttxts.append(acttxt)

## Text extraction and saving

In this section, we used the below code to extract all full texts from PDF files and save all act full texts into several format files including excel, csv, pickle and json.

In [ ]:
path = "your_download_path"
files = glob.glob(os.path.join(path, '*.pdf'))
acttxts = []
for file in files:
    doc = fitz.open(file)
    pgtxts = []
    for page in doc:
        pgtxt = page.get_text()
        pgtxts.append(pgtxt)
        acttxt = ' '.join(pgtxts)
    acttxts.append(acttxt)
    doc.close()

datasource = pd.DataFrame({
    'Full text': acttxts
})

# save bill info into files
datasource.to_excel('ND_Leginfo.xlsx')
datasource.to_csv('ND_Leginfo.csv')
datasource.to_pickle('ND_Leginfo.pkl')
datasource.to_json('ND_Leginfo.json')